In [1]:
!pip install newsapi-python

In [2]:
import os
import pandas as pd
from newsapi import NewsApiClient
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# nltk and vader imports

import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\josh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
api_key = "7a00abbd27124764b6d8b016bbeaf634"

#api_key = os.getenv(news_api)
api_key
print(api_key)

7a00abbd27124764b6d8b016bbeaf634


In [7]:
newsapi = NewsApiClient(api_key=api_key)

In [16]:
apple_news=newsapi.get_everything(q="Apple",language="en")

In [17]:
stock_apple_headlines = newsapi.get_top_headlines(q="Apple", language="en", country="us")
stock_apple_headlines 

{'status': 'ok',
 'totalResults': 1,
 'articles': [{'source': {'id': 'ars-technica', 'name': 'Ars Technica'},
   'author': 'Samuel Axon',
   'title': 'Apple announces dates and details about WWDC 2022 - Ars Technica',
   'description': 'Company will keep things virtual, as it has since the beginning of the pandemic.',
   'url': 'https://arstechnica.com/gadgets/2022/04/apple-announces-dates-and-details-about-wwdc-2022/',
   'urlToImage': 'https://cdn.arstechnica.net/wp-content/uploads/2022/04/Apple-WWDC22-announcement-hero-760x380.jpg',
   'publishedAt': '2022-04-05T18:43:41Z',
   'content': "Enlarge/ The image Apple shared alongside the WWDC 2022 announcement.\r\n13 with 12 posters participating\r\nApple's annual Worldwide Developers Conference (WWDC) will return this summer, as if there wer… [+1667 chars]"}]}

In [18]:
# Print total articles
print(f"Total articles about Apple: {stock_apple_headlines['totalResults']}")

# Show sample article
#stock_apple["articles"][0]

Total articles about Apple: 1


In [19]:
# Transform the response dictionary to a DataFrame
stock_apple_df = pd.DataFrame.from_dict(stock_apple_headlines["articles"])

stock_apple_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'ars-technica', 'name': 'Ars Technica'}",Samuel Axon,Apple announces dates and details about WWDC 2...,"Company will keep things virtual, as it has si...",https://arstechnica.com/gadgets/2022/04/apple-...,https://cdn.arstechnica.net/wp-content/uploads...,2022-04-05T18:43:41Z,Enlarge/ The image Apple shared alongside the ...


In [21]:
# Create the stock_x sentiment scores DataFrame
stock_apple_sentiments = []

for article in apple_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        stock_apple_sentiments.append({
            "text":text,
            "date":date,
            "compound":compound,
            "positive":pos,
            "neutral":neu,
            "negative":neg
        })
    except AttributeError:
        pass
    
stock_apple_df = pd.DataFrame(stock_apple_sentiments)
cols = ["date", "text", "compound", "positive", "neutral", "negative"]
stock_apple_df = stock_apple_df[cols]
stock_apple_df.head()

,date,text,compound,positive,neutral,negative
0,2022-03-21,Notice Apple that several Apple services are u...,0.1877,0.057,0.943,0.0
1,2022-03-11,"This week, Cherlynn and Devindra dive into all...",0.8591,0.183,0.817,0.0
2,2022-03-08,You're out of luck if you were hoping for a la...,0.7003,0.142,0.858,0.0
3,2022-03-29,Users report slow charging speeds after updati...,0.0000,0.000,1.000,0.0
4,2022-03-30,Apple is well-known for doing things in-house ...,0.1779,0.055,0.945,0.0


In [22]:
# Describe the stock_x Sentiment

stock_apple_df.describe()

,compound,positive,neutral,negative
count,20.000000,20.000000,20.000000,20.000000
mean,0.408765,0.103500,0.889350,0.007100
std,0.338087,0.093581,0.102805,0.022796
min,0.000000,0.000000,0.664000,0.000000
25%,0.048300,0.026250,0.849250,0.000000
50%,0.439300,0.091000,0.905000,0.000000
75%,0.669850,0.150750,0.973750,0.000000
max,0.924600,0.324000,1.000000,0.091000


In [24]:
stock_apple_df["positive"].mean()

0.10350000000000001

In [25]:
stock_apple_df["compound"].max()

0.9246

In [26]:
stock_apple_df["compound"].min()

0.0